<a href="https://colab.research.google.com/github/v-enigma/DL_LabExperiments/blob/main/lab8_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

3.Develop a sequence generator for Indian Classical Music Raga using an RNN to predict the
next note in a series. The notes involved are Sa, Re, Ga, Ma, Pa, Dha, Ni, and Sha.
Dataset Preparation: Create sequences of notes from the given raga scale (Sa, Re, Ga,
Ma, Pa, Dha, Ni, Sha).
Preprocess Data: Convert the notes to numerical representations.
Model Building: Build an RNN model to predict the next note in the sequence.
Training: Train the model to learn the relationships between the notes.
Generation: Use the trained model to generate sequences of notes.
As an extension, generate note sequences for Raga Bhairav, Raga Bhopali, Raga Bageshree, and
other ragas using the RNN model.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Define the notes in Indian Classical Music
# Sa, Re, Ga, Ma, Pa, Dha, Ni, Sha
notes = ['S', 'R', 'G', 'M', 'P', 'D', 'N', 'S\'']
note_to_int = {note: i for i, note in enumerate(notes)}
int_to_note = {i: note for i, note in enumerate(notes)}

# Define different ragas (patterns of notes)
ragas = {
    'Bhairav': ['S', 'R', 'G', 'M', 'P', 'D', 'N', 'S\''],  # All notes
    'Bhopali': ['S', 'R', 'G', 'P', 'D', 'S\''],             # Pentatonic
    'Bageshree': ['S', 'R', 'G', 'M', 'P', 'D', 'N', 'S\''], # All notes but specific patterns
    'Yaman': ['S', 'R', 'G', 'M', 'P', 'D', 'N', 'S\''],     # All notes
    'Bhairavi': ['S', 'R', 'G', 'M', 'P', 'D', 'N', 'S\'']   # All notes
}

# Generate sample sequences for each raga
def generate_raga_sequences(raga_name, num_sequences=50, seq_length=10):
    raga_notes = ragas[raga_name]
    sequences = []

    # For real applications, this would use authentic patterns
    # For this example, we'll generate somewhat random sequences based on the raga's note set
    for _ in range(num_sequences):
        # Create sequences with higher probability for characteristic movements
        if raga_name == 'Bhairav':
            # Bhairav often emphasizes the flat Re and Dha
            seq = []
            for _ in range(seq_length):
                if len(seq) == 0:
                    # Start with Sa more often
                    seq.append(np.random.choice(['S', 'P'], p=[0.8, 0.2]))
                else:
                    # Characteristic movements for Bhairav
                    if seq[-1] == 'S':
                        seq.append(np.random.choice(['R', 'N', 'S'], p=[0.6, 0.3, 0.1]))
                    elif seq[-1] == 'R':
                        seq.append(np.random.choice(['S', 'G'], p=[0.5, 0.5]))
                    else:
                        seq.append(np.random.choice(raga_notes))
        elif raga_name == 'Bhopali':
            # Bhopali: pentatonic, no Ga and Ni
            seq = []
            for _ in range(seq_length):
                if len(seq) == 0:
                    seq.append(np.random.choice(['S', 'P'], p=[0.7, 0.3]))
                else:
                    next_notes = ['S', 'R', 'G', 'P', 'D', 'S\'']
                    seq.append(np.random.choice(next_notes))
        else:
            # Generic pattern for other ragas
            seq = [np.random.choice(raga_notes) for _ in range(seq_length)]

        sequences.append(seq)

    return sequences

# Preprocess data: convert sequences to numerical format
def preprocess_sequences(sequences, seq_length):
    X = []
    y = []

    for sequence in sequences:
        for i in range(len(sequence) - seq_length):
            # Input sequence
            input_seq = sequence[i:i+seq_length]
            # Target note (next note after the input sequence)
            target_note = sequence[i+seq_length]

            # Convert notes to integers
            input_seq_int = [note_to_int[note] for note in input_seq]
            target_int = note_to_int[target_note]

            X.append(input_seq_int)
            y.append(target_int)

    # Convert to numpy arrays
    X = np.array(X)
    # One-hot encode the outputs
    y = to_categorical(y, num_classes=len(notes))

    return X, y

# Build the RNN model
def build_model(seq_length, vocab_size):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=50, input_length=seq_length),
        LSTM(128, return_sequences=True),
        LSTM(128),
        Dense(vocab_size, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Generate new sequences using the trained model
def generate_sequence(model, seed_sequence, seq_length, num_notes=30, temperature=1.0):
    generated_sequence = seed_sequence.copy()

    for _ in range(num_notes):
        # Get the last seq_length notes as input
        input_seq = generated_sequence[-seq_length:]
        # Convert to integers
        input_seq_int = np.array([[note_to_int[note] for note in input_seq]])

        # Predict the next note
        prediction = model.predict(input_seq_int, verbose=0)[0]

        # Apply temperature to control randomness
        prediction = np.log(prediction) / temperature
        prediction = np.exp(prediction) / np.sum(np.exp(prediction))

        # Sample from the prediction
        next_note_int = np.random.choice(len(notes), p=prediction)
        next_note = int_to_note[next_note_int]

        # Add the predicted note to the sequence
        generated_sequence.append(next_note)

    return generated_sequence

# Visualize the training history
def plot_history(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    return plt

# Main function to train and generate sequences for multiple ragas
def train_and_generate_ragas(ragas_list, sequence_length=5, epochs=50):
    all_models = {}
    all_generated_sequences = {}

    for raga_name in ragas_list:
        print(f"\nProcessing Raga {raga_name}...")

        # Generate sequences for this raga
        sequences = generate_raga_sequences(raga_name, num_sequences=100, seq_length=sequence_length+1)

        # Prepare data for training
        all_X = []
        all_y = []

        for sequence in sequences:
            if len(sequence) >= sequence_length + 1:
                for i in range(len(sequence) - sequence_length):
                    X_seq = sequence[i:i+sequence_length]
                    y_note = sequence[i+sequence_length]

                    X_seq_int = [note_to_int[note] for note in X_seq]
                    y_int = note_to_int[y_note]

                    all_X.append(X_seq_int)
                    all_y.append(y_int)

        if not all_X:  # Skip if no valid sequences
            print(f"No valid sequences for {raga_name}, skipping...")
            continue

        X = np.array(all_X)
        y = to_categorical(all_y, num_classes=len(notes))

        # Train-test split
        split_idx = int(0.8 * len(X))
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]

        # Build and train the model
        model = build_model(sequence_length, len(notes))
        history = model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=32,
            verbose=0
        )

        # Generate new sequences
        seed_sequence = sequences[0][:sequence_length]  # Use the first sequence as seed
        generated = generate_sequence(model, seed_sequence, sequence_length, num_notes=20)

        # Store results
        all_models[raga_name] = model
        all_generated_sequences[raga_name] = generated

        # Print results
        print(f"Original seed: {' '.join(seed_sequence)}")
        print(f"Generated sequence: {' '.join(generated)}")

        # Evaluate model
        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
        print(f"Test accuracy: {accuracy:.4f}, Loss: {loss:.4f}")

    return all_models, all_generated_sequences

# Example usage
if __name__ == "__main__":
    # List of ragas to process
    ragas_to_process = ['Bhairav', 'Bhopali', 'Bageshree', 'Yaman', 'Bhairavi']

    # Train models and generate sequences
    trained_models, generated_sequences = train_and_generate_ragas(ragas_to_process, sequence_length=5, epochs=30)

    # Print generated sequences for each raga
    print("\nGenerated Sequences for each Raga:")
    for raga_name, sequence in generated_sequences.items():
        print(f"{raga_name}: {' '.join(sequence)}")

    # Example of how to use a specific model to generate more sequences
    print("\nGenerating additional sequences for Raga Bhairav:")
    bhairav_model = trained_models.get('Bhairav')
    if bhairav_model:
        seed = ['S', 'R', 'G', 'M', 'P']
        for i in range(3):
            generated = generate_sequence(bhairav_model, seed, 5, num_notes=15, temperature=0.8)
            print(f"Sequence {i+1}: {' '.join(generated)}")


Processing Raga Bhairav...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Original seed: S N D N D
Generated sequence: S N D N D N D M P R P S N G M R N S S G D P P N R
Test accuracy: 0.4000, Loss: 2.0296

Processing Raga Bhopali...
Original seed: S R G P R
Generated sequence: S R G P R D P S' D S' P S' S' R D S' G R D D P D S G S
Test accuracy: 0.0500, Loss: 2.2387

Processing Raga Bageshree...
Original seed: P N G D G
Generated sequence: P N G D G D D S' P S S G R R G R S' R N G G N S' N S'
Test accuracy: 0.1000, Loss: 3.4005

Processing Raga Yaman...
Original seed: S S' D N S'
Generated sequence: S S' D N S' M N P S' D D G S R P D D D M S R R R S' S'
Test accuracy: 0.1000, Loss: 2.5407

Processing Raga Bhairavi...
Original seed: S S' M S' S'
Generated sequence: S S' M S' S' S M S' S S S S N G D G P N G N P G G D D
Test accuracy: 0.1000, Loss: 3.1665

Generated Sequences for each Raga:
Bhairav: S N D N D N D M P R P S N G M R N S S G D P P N R
Bhopali: S R G P R D P S' D S' P S' S' R D S' G R D D P D S G S
Bageshree: P N G D G D D S' P S S G R R G R S' R N